## Классификация текста с использованием CNN
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PragmaticsLab/NLP-course-AMI/blob/dev/seminars/sem3_classification.ipynb)

Используя данные отзывов IMDB, построим CNN для классификации документов на позитивный и негативный классы.

Источник изложения: https://github.com/bentrevett/pytorch-sentiment-analysis

В предположении, что PyTorch уже установлен, поставим дополнительные модули и загрузим модель для токенизации:

In [0]:
!pip3 install https://download.pytorch.org/whl/cpu/torch-1.0.1.post2-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 67.1MB 2.5MB/s 
ERROR: torchvision 0.3.0 has requirement torch>=1.1.0, but you'll have torch 1.0.1.post2 which is incompatible.
  Found existing installation: torch 1.1.0
    Uninstalling torch-1.1.0:
      Successfully uninstalled torch-1.1.0


In [0]:
!pip install torchvision

     |████████████████████████████████| 748.9MB 28kB/s 
  Found existing installation: torch 1.0.1.post2
    Uninstalling torch-1.0.1.post2:
      Successfully uninstalled torch-1.0.1.post2


In [0]:
!pip install torchtext

In [0]:
!pip install spacy

In [0]:
#!python3.6 -m spacy download en
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [0]:
import spacy
#import en
en_nlp = spacy.load('en_core_web_sm')

Загрузим датасет и получим из него выборку:

In [0]:
import torch
from torchtext import data
from torchtext import datasets

print(torch.__version__)

SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField()

train_src, test = datasets.IMDB.splits(TEXT, LABEL)

1.2.0
downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 11.1MB/s]


Попробуем обучить простую CNN на векторах слов. С учётом того, что в коллекции 100К уникальных слов, и векторы получатся достаточно громоздкие, урежем коллекцию до 25К слов, для всех прочих заведя токен unk (unknown). Кроме того, разобьём обучающий сет на обучение и валидацию для настройки параметров.

In [0]:
import random
train, valid = train_src.split(random_state=random.seed(SEED))

In [0]:
TEXT.build_vocab(train, max_size=25000)
LABEL.build_vocab(train)

Выше в словаре, помимо отсечения 25К наиболее частых слов, также появятся два специальных токена - unk и pad (padding).

Теперь создадим батчи, предварительно отсортировав тексты по длине, что минимизировать вставки pad-токенов:

In [0]:
BATCH_SIZE = 16

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size=BATCH_SIZE, 
    sort_key=lambda x: len(x.text), 
    repeat=False)

Опишем функцию подсчёта accuracy, а также функции обучения и применения сети:

In [0]:
import torch.nn.functional as F

def binary_accuracy(preds, y):
    rounded_preds = torch.round(F.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train_func(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        
        predictions = model(batch.text.cuda()).squeeze(1)

        loss = criterion(predictions.float(), batch.label.float().cuda())
        acc = binary_accuracy(predictions.float(), batch.label.float().cuda())
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss
        epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate_func(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text.cuda()).squeeze(1)

            loss = criterion(predictions.float(), batch.label.float().cuda())
            acc = binary_accuracy(predictions.float(), batch.label.float().cuda())

            epoch_loss += loss
            epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Начнём с подготовки данных:

In [0]:
TEXT.build_vocab(train, max_size=25000, vectors="glove.6B.100d")
LABEL.build_vocab(train)


BATCH_SIZEBATCH_S  = 8

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size=BATCH_SIZE, 
    sort_key=lambda x: len(x.text), 
    repeat=False)

.vector_cache/glove.6B.zip: 862MB [00:44, 19.2MB/s]                           
100%|█████████▉| 399103/400000 [00:12<00:00, 31526.93it/s]

Для создания свёрточного слоя воспользуемся nn.Conv2d, in_channels в нашем случае один (текст), out_channels -- это число число фильтров и размер ядер всех фильтров. Каждый фильтр будет иметь размерность [n x размерность эмбеддинга], где n - размер обрабатываемой n-граммы.

Важно, что предложения имели длину не меньше размера самого большого из используемых фильтров (здесь это не страшно, поскольку в используемых данных нет текстов, состоящих из пяти и менее слов).

In [0]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_0 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[0], embedding_dim))
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[1], embedding_dim))
        self.conv_2 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[2], embedding_dim))
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        #x = [sent len, batch size]
        x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        embedded = self.embedding(x)
                
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
        return self.fc(cat)

Сейчас мы можем использовать только три различных фильтра, хотелось бы больше. Воспользуйтесь nn.ModuleList и перепишите класс сети для того, чтобы фильтров создавалось по количеству элементов в filter_sizes.

Cоздадим модель и загрузим предобученные эмбеддинги:


In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)

In [0]:
TEXT.build_vocab(train, max_size=25000, vectors="glove.6B.100d")
LABEL.build_vocab(train)

pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [0]:
BATCH_SIZE  = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size=BATCH_SIZE, 
    sort_key=lambda x: len(x.text), 
    repeat=False)

Используя определённые ранее функции, запустим обучение с оптимизатором Adam и оценим качество на валидации и тесте:

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.cuda()

In [0]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_func(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate_func(model, valid_iterator, criterion)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc*100:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc*100:.2f}%')

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 01, Train Loss: 0.503, Train Acc: 74.45%, Val. Loss: 0.343, Val. Acc: 85.63%
Epoch: 02, Train Loss: 0.309, Train Acc: 87.06%, Val. Loss: 0.307, Val. Acc: 87.02%
Epoch: 03, Train Loss: 0.219, Train Acc: 91.36%, Val. Loss: 0.312, Val. Acc: 87.30%
Epoch: 04, Train Loss: 0.148, Train Acc: 94.58%, Val. Loss: 0.290, Val. Acc: 88.78%
Epoch: 05, Train Loss: 0.088, Train Acc: 97.16%, Val. Loss: 0.304, Val. Acc: 89.29%


In [0]:
test_loss , test_acc = evaluate_func(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc*100:.2f}%')

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Test Loss: 0.322, Test Acc: 87.86%


# Аугментация данных

В нашем примере данные были сбалансированными, а как работать с небалансированными данными. 

Рассмотрим задачу распознавания тональности твитов, взятых из [Twitter Sentimental Analysis challenge](https://datahack.analyticsvidhya.com/contest/practice-problem-twitter-sentiment-analysis/).

Источник изложения: https://github.com/mabusalah/Resampling

Получим данные

In [0]:
!wget https://www.dropbox.com/s/rxu4fud7xuyh7dv/test.csv
!wget https://www.dropbox.com/s/j6yh3xohvtqheqe/train_tweet.csv

--2019-09-15 18:48:21--  https://www.dropbox.com/s/rxu4fud7xuyh7dv/test.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/rxu4fud7xuyh7dv/test.csv [following]
--2019-09-15 18:48:23--  https://www.dropbox.com/s/raw/rxu4fud7xuyh7dv/test.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca2a47deb33d0df33292ed013bb.dl.dropboxusercontent.com/cd/0/inline/AokUZHx8FPR_lSqKNO8g_elmjGoAI3Ef0Vmkw_wYBCExQRHoTHvjTbA3e2fm6E3DnKGaLlw2RGP1RMCsMWSFnfTvk_H4pyQKZihwVhEJM2ou76kqeh2sB9t9uvLKC9qSnEs/file# [following]
--2019-09-15 18:48:23--  https://uca2a47deb33d0df33292ed013bb.dl.dropboxusercontent.com/cd/0/inline/AokUZHx8FPR_lSqKNO8g_elmjGoAI3Ef0Vmkw_wYBCExQRHoTHvjTbA3e2fm6E3DnKGaLlw2RGP1RMCsMWSFnfTvk_H4pyQKZihwVhEJM2ou76kqeh2sB9t9uvL

In [0]:
import pandas as pd
test = pd.read_csv('test.csv')
print("Test Set:"% test.columns, test.shape, len(test))
train = pd.read_csv('train_tweet.csv')
print("Training Set:"% train.columns, train.shape, len(train))

Test Set: (17197, 2) 17197
Training Set: (31962, 3) 31962


In [0]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [0]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


Итак, посмотрим, какой процент от общей выборки занимают позитивные и негативные примеры.

In [0]:
print("Positive: ", train.label.value_counts()[0]/len(train)*100,"%")
print("Negative: ", train.label.value_counts()[1]/len(train)*100,"%")

Positive:  92.98542018647143 %
Negative:  7.014579813528565 %


93% vs. 7% - данные определенно несбалансированны, что, в свою очередь, негативно влияет на точность предсказания.
Для начала поработаем с исходными данными и оценим точность классификации.
Начнем с предобработки данных: уберем из твитов числа, html/xml-тэги, специальные символы.

In [0]:
import re
from bs4 import BeautifulSoup #для работы с html/xml-тэгами
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import PorterStemmer

porter=PorterStemmer()
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()

    words = tok.tokenize(lower_case)
    
    stem_sentence=[]
    for word in words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    words="".join(stem_sentence).strip()
    return words

nums = [0,len(train)]
clean_tweet_texts = []
for i in range(nums[0],nums[1]):
    clean_tweet_texts.append(tweet_cleaner(train['tweet'][i]))
    
nums = [0,len(test)]
test_tweet_texts = []

for i in range(nums[0],nums[1]):
    test_tweet_texts.append(tweet_cleaner(test['tweet'][i])) 
    
train_clean = pd.DataFrame(clean_tweet_texts,columns=['tweet'])
train_clean['label'] = train.label
train_clean['id'] = train.id
test_clean = pd.DataFrame(test_tweet_texts,columns=['tweet'])
test_clean['id'] = test.id

Разделим данные на обучающие и проверочные.

In [0]:
from sklearn import model_selection, preprocessing, metrics, linear_model, svm

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train_clean['tweet'],train_clean['label'])
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

Рассчитаем TF-IDF признаки.

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100000)
tfidf_vect.fit(train_clean['tweet'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

Попробуйте использовать обычный счетчик слов для извлечения признаков.

Точность в качестве метрики работает хорошо только на сбалансированных наборах данных, поэтому для оценки результатов работы  алгоритма будем использовать F1-метрику.

In [0]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    classifier.fit(feature_vector_train, label)

    predictions = classifier.predict(feature_vector_valid)    

    return metrics.f1_score(valid_y,predictions)

Для начала обучим лог-регрессию.

In [0]:
accuracyORIGINAL = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),xtrain_tfidf, train_y, xvalid_tfidf)
print ("Logistic regression Baseline, WordLevel TFIDF: ", accuracyORIGINAL)

Logistic regression Baseline, WordLevel TFIDF:  0.5136612021857924


Как видно, результат оставляет желать лучшего.

Что можно сделать с данными?

Было бы неплохо как-то увеличить  количество негативных примеров, или же уменьшить количество положительных. Для этого существуют различные техники аугментации данных. 
В Python для этих целей есть библиотека imblearn (imbalanced-learn).

In [0]:
from imblearn.over_sampling import BorderlineSMOTE, SMOTE, ADASYN, SMOTENC, RandomOverSampler
from imblearn.under_sampling import (RandomUnderSampler, 
                                    NearMiss, 
                                    InstanceHardnessThreshold,
                                    CondensedNearestNeighbour,
                                    EditedNearestNeighbours,
                                    RepeatedEditedNearestNeighbours,
                                    AllKNN,
                                    NeighbourhoodCleaningRule,
                                    OneSidedSelection,
                                    TomekLinks)
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.pipeline import make_pipeline

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


Итак, в качестве инструментов для аугментации рассмотрим: under-sampling, over-sampling и их комбинацию.

**Under-sampling** уравновешивает данные за счет уменьшения размера  превалирующего класса.
Этот метод разумно использовать, когда количество данных достаточно велико, иначе есть риск остаться и вовсе без обучающих примеров.

Итак, логика действия довольно проста: мы просто случайным образом убираем лишние экземпляры из превалирующего класса.

Так как в нашем примере лишь 7% всех твитов имеют негативную окраску, уравновешивание позитивного набора с этими 7-ю процентами вряд ли обеспечит хороший результат.

Попробуем...

In [0]:
rus = RandomUnderSampler(random_state=0, replacement=True)
rus_xtrain_tfidf, rus_train_y = rus.fit_sample(xtrain_tfidf, train_y)
accuracyrus = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),rus_xtrain_tfidf, rus_train_y, xvalid_tfidf)
print ("Logistic regressio RUS, WordLevel TFIDF: ", accuracyrus)

Logistic regressio RUS, WordLevel TFIDF:  0.4744842562432139


Действительно, все стало только хуже.

Попробуем другие алгоритмы **under-sampling**.

Например, **NearMiss**. Данный алгоритм выбирает, какие экземпляры нужно оставить в превалирующем классе на основании некоторых эвристик. Существует три варианта данного алгоритма:

**NearMiss-1** оставляет те экземпляры из превалирующего класса, для которых среднее расстояние до *k* ближайших соседей из миноритарного класса будет наименьшим.

**NearMiss-2** оставляет те экземпляры из превалирующего класса, для которых среднее расстояние до *k* самых дальних соседей из миноритарного класса будет наименьшим.

**NearMiss-3** состоит из двух шагов: сначала, для каждого экземпляра из миноритарного класса выбирается *k* ближайших соседей из превалирующего класса, затем, из большего класса выбираются те экземпляры, для которых среднее расстояние до *k* ближайших соседей максимальное.

In [0]:
for sampler in (NearMiss(version=1),NearMiss(version=2),NearMiss(version=3)):
    nm_xtrain_tfidf, nm_train_y = sampler.fit_sample(xtrain_tfidf, train_y)
    accuracysm = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),nm_xtrain_tfidf, nm_train_y, xvalid_tfidf)
    print ("Logistic regression NearMiss(version= {0}), WordLevel TFIDF: ".format(sampler.version), accuracysm)

Logistic regression NearMiss(version= 1), WordLevel TFIDF:  0.25690140845070425
Logistic regression NearMiss(version= 2), WordLevel TFIDF:  0.49223946784922396


/usr/local/lib/python3.6/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:194: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn('The number of the samples to be selected is larger'


Logistic regression NearMiss(version= 3), WordLevel TFIDF:  0.2904228855721393


**Edited Nearest Neighbor (ENN)**

ENN удаляет из большего класса элемент, если класс его ближайшего соседа отличается от его собственного.

In [0]:
enn_xtrain_tfidf, enn_train_y = EditedNearestNeighbours().fit_sample(xtrain_tfidf, train_y)
accuracy = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),enn_xtrain_tfidf, enn_train_y, xvalid_tfidf)
print ("Logistic regression {0}, WordLevel TFIDF: ", accuracy)

Logistic regression {0}, WordLevel TFIDF:  0.5254691689008043


Как вы поняли, при применении **Under-samplin**g техник новые данные не генерируются, в отличие от **Over-sampling**.

# Over-sampling

Итак, когда данных недостаточно или количество экземпляров в миноритарном классе очень мало применяется **Over-sampling**. 

При применении этой техники балансировка данных происходит за счет увеличения количества экземпляров в миноритарном классе. Новые элементы генерируются за счет: повторения, бутстрэппинга, SMOTE (Synthetic Minority Over-Sampling Technique) или ADASYN (Adaptive synthetic sampling).

**Random Over-sampling**: случайным образом дублируются некоторые элементы из миноритарного класса.

In [0]:
#Random Over Sampling
ros = RandomOverSampler(random_state=777)
ros_xtrain_tfidf, ros_train_y = ros.fit_sample(xtrain_tfidf, train_y)
accuracyROS = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),ros_xtrain_tfidf, ros_train_y, xvalid_tfidf)
print ("Logistic regression ROS, WordLevel TFIDF: ", accuracyROS)

Logistic regression ROS, WordLevel TFIDF:  0.6431034482758621


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


**SMOTE Over-sampling**

Алгоритм SMOTE основан на идее генерации некоторого количества искусственных примеров, которые были бы «похожи» на имеющиеся в миноритарном классе, но при этом не дублировали их.

Для создания новой записи находят разность $d=X_b-X_a,$ где $ X_b, X_a -$ векторы признаков «соседних» примеров $a$ и $b$ из миноритарного класса. 

Их находят, используя алгоритм ближайшего соседа (*KNN*). В данном случае необходимо и достаточно для примера $b$ получить набор из $k$ соседей, из которого в дальнейшем будет выбрана запись $b$. Остальные шаги алгоритма *KNN* не требуются.

Далее из $d$ путем умножения каждого его элемента на случайное число в интервале (0, 1) получают $\hat{d}$. Вектор признаков нового примера вычисляется путем сложения $X_a$ и $\hat{d}$. 

Алгоритм **SMOTE** позволяет задавать количество записей, которое необходимо искусственно сгенерировать. Степень сходства примеров $a$ и $b$ можно регулировать путем изменения значения $k$ (числа ближайших соседей).

In [0]:
sm = SMOTE(random_state=777, ratio = 1.0)
sm_xtrain_tfidf, sm_train_y = sm.fit_sample(xtrain_tfidf, train_y)
accuracySMOTE = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),sm_xtrain_tfidf, sm_train_y, xvalid_tfidf)
print ("Logistic regression SMOTE, WordLevel TFIDF: ", accuracySMOTE)

Logistic regression SMOTE, WordLevel TFIDF:  0.6519434628975266


Итак, по сравнению с **Random Over-sampling** разница небольшая.

Проверьте результаты **Random Over-sampling** и **SMOTE Over-sampling** для реальных тестовых данных (*test_clean*).

Следующий алгоритм **ASMO: Adaptive synthetic minority oversampling**.



Сгенерировать искусственные записи в пределах отдельных кластеров на основе всех классов. Для каждого примера миноритарного класса находят m ближайших соседей, и на основе них (также как в SMOTE) создаются новые записи.

1.   Если для каждого $i$-ого примера миноритарного класса из $k$ ближайших соседей $g$ ($g\leq k$) принадлежит к мажоритарному, то набор данных считается «рассеянным». В этом случае используют алгоритм **ASMO**, иначе применяют **SMOTE** (как правило, $g$ задают равным 20).
2.   Используя только примеры миноритарного класса, выделить несколько кластеров (например, алгоритмом $k$-means).
3.   Сгенерировать искусственные записи в пределах отдельных кластеров на основе всех классов. Для каждого примера миноритарного класса находят m ближайших соседей, и на основе них (также как в **SMOTE**) создаются новые записи.

Такая модификация алгоритма **SMOTE** делает его более адаптивным к различным наборам данных с несбалансированными классами.

In [0]:
ad = ADASYN(random_state=777, ratio = 1.0)
ad_xtrain_tfidf, ad_train_y = ad.fit_sample(xtrain_tfidf, train_y)
accuracyADASYN = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),ad_xtrain_tfidf, ad_train_y, xvalid_tfidf)
print ("Logistic regression ADASYN, WordLevel TFIDF: ", accuracyADASYN)

Logistic regression ADASYN, WordLevel TFIDF:  0.6495726495726495


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


И опять проверим на реальных тестовых примерах.

# Комбинация **Under-** и **Over-sampling**

В *imblearn* реализованы две возможные комбинации:


1.   **SMOTE** + **ENN**
2.   **SMOTE** + **Tomek Link Removal** (Пара двух ближайших соседей, которые принадлежат разным классам называется *Tomek link*. Under-sampling заключается в удалении всех таких элементов из мажоритарного класса)

Подробнее: https://imbalanced-learn.readthedocs.io/en/stable/api.html#module-imblearn.combine



In [0]:
se = SMOTEENN(random_state=42)
se_xtrain_tfidf, se_train_y = se.fit_sample(xtrain_tfidf, train_y)
accuracy = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),se_xtrain_tfidf, se_train_y, xvalid_tfidf)
print ("Logistic regression SMOTEENN: ", accuracy)

Logistic regression SMOTEENN:  0.487778381314503


Первый метод сработал плохо. Оцените работу второго подхода.